## Sentiment Tool

In [72]:
from pipeline import NycNtaLoader, TwitterIngestor
from sentiment import NTA
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import math

### Get New York City NTAs

In [87]:
nta_loader = NycNtaLoader()
keywords = nta_loader.load(get_keywords=True)
df = nta_loader.load()
df['Keyword'] = keywords

sentiment_analysis = []

for idx, row in df.iterrows():
    nta = NTA(nta=row['Keyword'])
    sentiment_df = nta.get_nta_sentiment_df()
    
    buckets = np.arange(-1, 1.2, 0.2)
    buckets = [(round(buckets[i], 2), round(buckets[i+1], 2)) for i in range(len(buckets) - 1)]
    sentiment_count = np.array([0 for bucket in buckets])
    for sentiment in sentiment_df['vader_sent'].to_numpy():
        for i in range(len(buckets)):
            if sentiment > buckets[i][0] and sentiment <= buckets[i][1]:
                sentiment_count[i] += 1
    sentiment_pct = list(sentiment_count / sum(sentiment_count))

    sentiment_dict = {}
    sentiment_dict['district'] = row['NTA']
    sentiment_dict['sentiment'] = np.mean(sentiment_df['vader_sent'].to_numpy())
    sentiment_dict['histogram'] = sentiment_pct
    
    if sentiment_dict['district'] == 'East New York':
        continue
    if math.isnan(sentiment_dict['sentiment']):
        sentiment_dict['sentiment'] = 'N/A'
    for i in range(len(sentiment_dict['histogram'])):
        if math.isnan(sentiment_dict['histogram'][i]):
            sentiment_dict['histogram'][i] = 'N/A'
    
    sentiment_analysis.append(sentiment_dict)

sentiment_analysis = sorted(sentiment_analysis, key=lambda k: k['district']) 

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:19: RuntimeWarning: invalid value encountered in true_divide


In [88]:
import json
with open('sentiment.json', 'w') as f:
    json.dump(sentiment_analysis, f)